# Stanford SQL Course

This are personal notes about the SQL course offered by Stanford Lagunita.

The url of the course is https://lagunita.stanford.edu/courses/DB/SQL/SelfPaced/course/

In [6]:
# Showing content of repo
!ls

colleges.db  readme.md	stanford_sql_notes.Rmd	stanford_sql_notes.ipynb


In [7]:
# Importing neccessary libraries
import sqlite3
import pandas as pd

In [8]:
# Writting helper functions

db = 'colleges.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    return run_query('''
    select *
    from sqlite_master
    where type in ('table', 'view')
    ''')


In [9]:
# Showing tables
show_tables()

,type,name,tbl_name,rootpage,sql
0,table,College,College,2,"CREATE TABLE College(cName text, state text, e..."
1,table,Student,Student,3,"CREATE TABLE Student(sID int, sName text, GPA ..."
2,table,Apply,Apply,4,"CREATE TABLE Apply(sID int, cName text, major ..."


## Basic Select Statements

In [10]:
# Example of the basic select statement

run_query('''
    select sID, sName, GPA
    from Student
    where GPA > 3.6
''')

,sID,sName,GPA
0,123,Amy,3.9
1,456,Doris,3.9
2,678,Fay,3.8
3,987,Helen,3.7
4,876,Irene,3.9
5,654,Amy,3.9


In [11]:
# Using a select statement to perform an inner join

run_query('''
    select sName, major
    from Student, Apply
    where Student.sID = Apply.sID
''')

,sName,major
0,Amy,CS
1,Amy,CS
2,Amy,EE
3,Amy,EE
4,Bob,biology
5,Craig,CS
6,Craig,EE
7,Craig,bioengineering
8,Craig,bioengineering
9,Fay,history


In [12]:
# Using the 'distinct' keyword to get only unique results in the obtained relation

run_query('''
    select distinct sName, major
    from Student, Apply
    where Student.sID = Apply.sID
''')

,sName,major
0,Amy,CS
1,Amy,EE
2,Bob,biology
3,Craig,CS
4,Craig,EE
5,Craig,bioengineering
6,Fay,history
7,Helen,CS
8,Irene,CS
9,Irene,biology


In [13]:
# Use of the "and" operator to specify multiple conditions on a 'where' clause

run_query('''
    select sName, GPA, decision
    from Student, Apply
    where Student.sID = Apply.sID
        and sizeHS < 1000
        and major = 'CS'
        and cname = 'Stanford';
''')

,sName,GPA,decision
0,Helen,3.7,Y
1,Irene,3.9,N


In [14]:
# When performing joins is important to check any ambiguous column names

run_query('''
    select College.cName 
    from College, Apply
    where College.cName = Apply.cName
        and enrollment > 20000
        and major = 'CS'
''')

,cName
0,Berkeley
1,Cornell
2,Berkeley


In [15]:
# Joining three relations were table A is related to table B, and table B is related to table C

run_query('''
    select sName, GPA, Apply.cName
    from Student, College, Apply
    where Student.sID = Apply.sID and Apply.cName = College.cName;
''')

,sName,GPA,cName
0,Amy,3.9,Berkeley
1,Amy,3.9,Cornell
2,Amy,3.9,Stanford
3,Amy,3.9,Stanford
4,Bob,3.6,Berkeley
5,Craig,3.5,Cornell
6,Craig,3.5,Cornell
7,Craig,3.5,Cornell
8,Craig,3.5,MIT
9,Fay,3.8,Stanford


In [16]:
# Order the results of a query using the 'order by' clause and the 'desc'keyword

# When ordering by two different attributes, the first given will define the first sort
# and the second attribute will be applied afterwards
run_query('''
    select sName, GPA, Apply.cName
    from Student, College, Apply
    where Student.sID = Apply.sID and Apply.cName = College.cName
    order by sName desc, GPA
''')

,sName,GPA,cName
0,Jay,2.9,Cornell
1,Jay,2.9,Cornell
2,Jay,2.9,Stanford
3,Irene,3.9,MIT
4,Irene,3.9,MIT
5,Irene,3.9,Stanford
6,Helen,3.7,Berkeley
7,Helen,3.7,Stanford
8,Fay,3.8,Stanford
9,Craig,3.4,MIT


In [17]:
# The 'like' operator can be used inside a 'where' clause to select 

run_query('''
    select major, sID
    from Apply
    where major like '%eng%'
''')


,major,sID
0,bioengineering,345
1,bioengineering,345


In [18]:
# On the select clause it's also possible to perform arithmetic operations

run_query('''
    select gpa * 100
    from Student;

''')

,gpa * 100
0,390.0
1,360.0
2,350.0
3,390.0
4,290.0
5,380.0
6,340.0
7,370.0
8,390.0
9,290.0


In [19]:
# You can use aliases to rename any attribute on the resulting relation

run_query('''
    select gpa * 100 as upgraded_gpa
    from Student;

''')

,upgraded_gpa
0,390.0
1,360.0
2,350.0
3,390.0
4,290.0
5,380.0
6,340.0
7,370.0
8,390.0
9,290.0


## Table Variables and Set Operators

In [23]:
# Table variables are simply aliases used on relations. This means that this aliases are 
# declared on the "from" statement.

run_query('''
    select S.sID, A.cName, enrollment
    from Student as S, College as C, Apply as A
    where A.sID = S.sID and A.cName = C.cName;
''')

,sID,cName,enrollment
0,123,Berkeley,36000
1,123,Cornell,21000
2,123,Stanford,15000
3,123,Stanford,15000
4,234,Berkeley,36000
5,345,Cornell,21000
6,345,Cornell,21000
7,345,Cornell,21000
8,345,MIT,10000
9,678,Stanford,15000


In [24]:
# The union operator (union) is used to stack two relations. Each relation might be a table or can also be the product
# of an statement

run_query('''
    select cName from College
    
    union
    
    select sName from Student;
''')

,cName
0,Amy
1,Berkeley
2,Bob
3,Cornell
4,Craig
5,Doris
6,Edward
7,Fay
8,Gary
9,Helen


In [25]:
# When performing unions, SQL will automatically delete the duplicates. To change this behaviour, you 
# can se the "all" keyword

run_query('''
    select cName from College
    
    union all
    
    select sName from Student;
''')

,cName
0,Stanford
1,Berkeley
2,MIT
3,Cornell
4,Amy
5,Bob
6,Craig
7,Doris
8,Edward
9,Fay


In [33]:
# The intersect operator returns the rows that appear in two relations. 

run_query('''
    select sID from Apply where major = 'CS'
    
    intersect
    
    select sID from Apply where major = 'EE';
''')

,sID
0,123
1,345


In [30]:
# The intersect operator don't add expressive power to the language,
# Any statement made with the intersect operator could also be performed with a self join

run_query('''
    select distinct A1.sID
    from Apply as A1, Apply as A2
    where A1.sID = A2.sID and A1.major = 'CS' and A2.major = 'EE'
''')

,sID
0,123
1,345


In [31]:
# The except operator returns every row in a first relation that is not in a second relation

run_query('''
    select sID from Apply where major = 'CS'
    
    except
    
    select sID from Apply where major = 'EE'
''')

,sID
0,543
1,876
2,987


In [44]:
# Again, the except operator don't add any expressive power to SQL
# The same query can be written using an inner join and which statements

run_query('''
    select distinct sID
    from Apply
    where Apply.major = 'CS' and sID not in 
        (select distinct A1.sID
            from Apply as A1, Apply as A2
            where A1.sID = A2.sID and A1.major = 'CS' and A2.major = 'EE')
''')

,sID
0,987
1,876
2,543


## Subqueries in the WHERE Clause


In [45]:
# Subqueries can be used in conjunction with the "in" operator to check if which values are in another computed
# relation

run_query('''
    select sID, sName
    from Student
    where sID in (select sID from Apply where major = 'CS');
''')

,sID,sName
0,123,Amy
1,345,Craig
2,987,Helen
3,876,Irene
4,543,Craig


In [48]:
# You can also write a query using "in" and "not in" operators to emmulate the way the except operator works

run_query('''
select distinct sID
from Apply
where sID in (select sID from Apply where major = 'CS')
    and sID not in (select sID from Apply where major = 'EE')
''')

,sID
0,987
1,876
2,543


In [51]:
# The "exists" operator is used to check if a subquery is empty or not
# Here, there is also a "correlated reference". This means that the inner query is accesing a value
# from the outter query

run_query('''
    select cName, state
    from College C1
    where exists (select * from College C2
                    where C2.state = C1.state and C2.cName != C1.cName);
''')

# This query is computing the subquery for each of the rows in the outer query, this is, for each college name
# SQL computes the relation of the colleges in the same state and with a different name. If there is at least one
# row in this relation, the statement will be evaluated as True

,cName,state
0,Stanford,CA
1,Berkeley,CA


In [53]:
# The "not exists" operator returns True when a Query is empty. This functionality can be used 
# to emmulate the selection of a max or min value

run_query('''
select cName
from College C1
where not exists (select *
                    from College C2
                    where C2.enrollment > C1.enrollment)
''')

# This query takes each college and compute a subquery that returns a relation containing Colleges with higher number of
# enrollments. If there is no college with a higher number of enrollments, then the current college is the one with
# the biggest enrollment

,cName
0,Berkeley


In [55]:
# The "all" statement evaluates an inequality between a number and a relation made of numbers. If the inequality is
# true for all the cases when the number and the elements of the relation is compared, then True is returned.

run_query('''
select sName, GPA
from Student
where GPA >= all (select GPA from Student);
''')

# This query compares each of the GPA values against the list of all GPA's. If the current GPA is bigger or equal to all
# of them, the name of the student is returned, hence finding the student(s) with the higher GPA.

# NOTE: THE "all" OPERATOR DON'T EXIST IN SQL

DatabaseError: Execution failed on sql '
select sName, GPA
from Student
where GPA >= all (select GPA from Student);
': near "all": syntax error

In [56]:
# The "any" statement also evaluates an inequality between a value and a relation, but this time the result returns True
# when the relations holds for at least one of the rows

run_query('''
    select Cname
    from College S1
    where not enrollment <= any (select enrollment from College S2
                                    where S2.cName != S1.cName);

''')

# NOTE: THE 'any' OPERATOR DON'T EXIST IN SQL

DatabaseError: Execution failed on sql '
    select Cname
    from College S1
    where not enrollment <= any (select enrollment from College S2
                                    where S2.cName != S1.cName);

': near "select": syntax error

## Subqueries in FROM and SELECT

In [57]:
# The subqueries on the FROM clause are used to avoid performing multiple calculations 

run_query('''
    select *
    from ( select sID, sName, GPA, GPA*(sizeHS /1000)as scaledGPA
        from Student) as G
    where abs(G.scaledGPA - GPA) > 1
''')

,sID,sName,GPA,scaledGPA
0,345,Craig,3.5,0.0
1,567,Edward,2.9,5.8
2,678,Fay,3.8,0.0
3,789,Gary,3.4,0.0
4,987,Helen,3.7,0.0
5,876,Irene,3.9,0.0
6,543,Craig,3.4,6.8


In [63]:
# The subqueries on the SELECT clause are used to calculate a single value for each tupple of the 
# outer query using referenciation. 

run_query('''
    select cName, state,
        (select max(GPA)
        from Apply, Student
        where Apply.cName = College.cName) as GP
    from College
''')

,cName,state,GP
0,Stanford,CA,3.9
1,Berkeley,CA,3.9
2,MIT,MA,3.9
3,Cornell,NY,3.9


## The join family of operators

In [64]:
# The inner join is basically the same as the cartesian product and the 'where' clause together
# The only difference is the syntax (use of "on" instead of "where")

run_query('''
    select distinct cName, major
    from Student inner join Apply
    on Student.sID = Apply.sID
''')

,cName,major
0,Berkeley,CS
1,Cornell,EE
2,Stanford,CS
3,Stanford,EE
4,Berkeley,biology
5,Cornell,CS
6,Cornell,bioengineering
7,MIT,bioengineering
8,Stanford,history
9,MIT,biology


In [65]:
# the default type of join is the inner join, so if no join is specified, the inner join will be 
# performed

run_query('''
    select distinct cName, major
    from Student join Apply
    on Student.sID = Apply.sID
''')

,cName,major
0,Berkeley,CS
1,Cornell,EE
2,Stanford,CS
3,Stanford,EE
4,Berkeley,biology
5,Cornell,CS
6,Cornell,bioengineering
7,MIT,bioengineering
8,Stanford,history
9,MIT,biology


In [67]:
# the 'on' clause can alse be used to specify other conditions that would be placed on a where statement
# otherwise

run_query('''
    select sName, GPA
    from Student join Apply
    on Student.sID = Apply.sID
    and sizeHS < 1000 and major = 'CS' and cName = 'Stanford';
''')

,sName,GPA
0,Helen,3.7
1,Irene,3.9


In [69]:
# The natural join will automatically perform a join in the attributes of both tables that have the same name
# It will also automatically delete the repeated attribute (in this case the sID)
run_query('''
    select distinct *
    from Student natural join Apply;
''')

,sID,sName,GPA,sizeHS,cName,major,decision
0,123,Amy,3.9,1000,Berkeley,CS,Y
1,123,Amy,3.9,1000,Cornell,EE,Y
2,123,Amy,3.9,1000,Stanford,CS,Y
3,123,Amy,3.9,1000,Stanford,EE,N
4,234,Bob,3.6,1500,Berkeley,biology,N
5,345,Craig,3.5,500,Cornell,CS,Y
6,345,Craig,3.5,500,Cornell,EE,N
7,345,Craig,3.5,500,Cornell,bioengineering,N
8,345,Craig,3.5,500,MIT,bioengineering,Y
9,678,Fay,3.8,200,Stanford,history,Y


In [76]:
# It's considered a good practice to perform natural joins specifying the attribute that will be used
# using the 'using' statement

run_query('''
    select sName, GPA
    from Student natural join Apply using(sID)
    where sizeHS < 1000 and major = 'CS' and cName = 'Stanford';
''')

# The SQLite engine don't suppor the 'using' operator on natural joins

DatabaseError: Execution failed on sql '
    select sName, GPA
    from Student natural join Apply using(sID)
    where sizeHS < 1000 and major = 'CS' and cName = 'Stanford';
': a NATURAL join may not have an ON or USING clause

In [81]:
# The left join works as usual

run_query('''
    select sName, sID, major
    from Student
    left outer join Apply using(sID);
''')

# Note that SQLite will mark a None where there is no corresponding information on the right table

,sName,sID,major
0,Amy,123,CS
1,Amy,123,CS
2,Amy,123,EE
3,Amy,123,EE
4,Bob,234,biology
5,Craig,345,CS
6,Craig,345,EE
7,Craig,345,bioengineering
8,Craig,345,bioengineering
9,Doris,456,None


In [88]:
# The outter joins could also be rewritter using the union operator 

run_query('''
    select sName, sID, major
    from Student
    natural join Apply
    where Student.sID = Apply.sID
    
    union all
    
    select sName, sID, Null
    from Student
    where sID not in (select sID from Apply)
''')

# Here, the students that do apply are the joined using the union operator with the students that don't appear
# on the Apply table (i.e. didn't apply to any college)

,sName,sID,major
0,Amy,123,CS
1,Amy,123,CS
2,Amy,123,EE
3,Amy,123,EE
4,Bob,234,biology
5,Craig,345,CS
6,Craig,345,EE
7,Craig,345,bioengineering
8,Craig,345,bioengineering
9,Fay,678,history


In [89]:
# You could also use the natural type of join in conjuction with the left join
# This will automatically use the attributes with the same name to perform the left join

run_query('''
    select sName, sID, cName, major
    from Apply
    natural left outer join Student;
''')

,sName,sID,cName,major
0,Amy,123,Stanford,CS
1,Amy,123,Stanford,EE
2,Amy,123,Berkeley,CS
3,Amy,123,Cornell,EE
4,Bob,234,Berkeley,biology
5,Craig,345,MIT,bioengineering
6,Craig,345,Cornell,bioengineering
7,Craig,345,Cornell,CS
8,Craig,345,Cornell,EE
9,Fay,678,Stanford,history


In [90]:
# The right join works in the same way as the left join

run_query('''
    select sName, sID, major
    from Student
    right outer join Apply using(sID);
''')

# Note. right joins are not supported

DatabaseError: Execution failed on sql '
    select sName, sID, major
    from Student
    right outer join Apply using(sID);
': RIGHT and FULL OUTER JOINs are not currently supported

In [91]:
# The full outer join also works, but is not supported on SQL

run_query('''
    select sName, sID, major
    from Student
    full outer join Apply using(sID);
''')

DatabaseError: Execution failed on sql '
    select sName, sID, major
    from Student
    full outer join Apply using(sID);
': RIGHT and FULL OUTER JOINs are not currently supported